In [1]:
import os,sys
 

os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["PYSPARK_PYTHON"]        = sys.executable


In [2]:
from pyspark import SparkContext
from sympy import *
from drudge import *
from gristmill import *
print("finished importing")

finished importing


In [3]:
import re

# ----------------------------------------------------------------------------
# 1) Start Spark & BCS quasiparticle drudge
# ----------------------------------------------------------------------------
ctx = SparkContext('local[2]', 'bcs_ccsd')
dr  = ReducedBCSDrudge(ctx)
dr.full_simplify = True
print("finished")

25/11/04 21:43:58 WARN Utils: Your hostname, Swarnamoys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.87 instead (on interface en0)
25/11/04 21:43:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/04 21:43:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

finished


In [5]:
p,q,r,s = dr.names.A_dumms[:4]
P, Pdag, N = dr.names.P, dr.names.Pdag, dr.names.N
u, v = IndexedBase('u'), IndexedBase('v')

In [7]:
H120, H110 , H021 = IndexedBase('h120'), IndexedBase('h110'), IndexedBase('h021')
H011, H110 = IndexedBase('h011'), IndexedBase('h110')
H100, H011 = IndexedBase('h100'), IndexedBase('h011')
H001, H020 = IndexedBase('h001'), IndexedBase('h020')
H010 = IndexedBase('h010') 
H000 = IndexedBase('h000')

P, Pdag, N = dr.names.P, dr.names.Pdag, dr.names.N
P_i_dag = (u[p]*v[p]+u[p]**2*Pdag[p]
     - u[p]*v[p]*N[p]
     - v[p]**2*P[p])

P_j_dag = (
        u[q]*v[q]
      + u[q]**2 * Pdag[q]
      - u[q]*v[q] * N[q]
      - v[q]**2 * P[q]
    )

P_k_dag = (
        u[r]*v[r]
      + u[r]**2 * Pdag[r]
      - u[r]*v[r] * N[r]
      - v[r]**2 * P[r]
    )
P_i = (
        (u[p])*(v[p])
      + (u[p])**2 * P[p]
      - (u[p])*(v[p]) * N[p]
      - (v[p])**2 * Pdag[p]
    )
P_j = ((u[q])*(v[q])
      + (u[q])**2 * P[q]
      - (u[q])*(v[q]) * N[q]
      - (v[q])**2 * Pdag[q])

P_k = ((u[r])*(v[r])
      + (u[r])**2 * P[r]
      - (u[r])*(v[r]) * N[r]
      - (v[r])**2 * Pdag[r])
#hc = P_j_dag * P_i
N_i = 2*(v[p])*v[p] +((u[p])*u[p] - (v[p])*v[p])*N[p] +2*u[p]*(v[p])*Pdag[p]\
            +2*(u[p])*v[p]*P[p]
N_j = 2*(v[q])*v[q] +((u[q])*u[q] - (v[q])*v[q])*N[q] +2*u[q]*(v[q])*Pdag[q]\
            +2*(u[q])*v[q]*P[q]

N_k = 2*(v[r])*v[r] +((u[r])*u[r] - (v[r])*v[r])*N[r] +2*u[r]*(v[r])*Pdag[r]\
       +2*(u[r])*v[r]*P[r]

expr = H120[p,q,r]*Pdag[p]*N[q]*N[r]  + H021[p,q,r]*N[p]*N[q]*P[r]  + H110[p,q]*Pdag[p]*N[q]\
 + H100[p]*Pdag[p] + H011[p,q]*N[p]*P[q] + H001[p]*P[p] + H010[p]*N[p] + H020[p,q]*N[p]*N[q]
expr = dr.einst(expr) 
expr.display()

<IPython.core.display.Math object>

In [8]:
dr.set_symm(H021, Perm([1, 0,2],IDENT), valence=3) #This is correct, the symmetry factor is already 
dr.set_symm(H120, Perm([0, 2,1],IDENT), valence=3)
dr.set_symm(H020, Perm([1,0], IDENT),valence=2)

expr = H120[p,q,r]*P_i_dag*N_j*N_k  + H021[p,q,r]*N_i*N_j*P_k  + H110[p,q]*P_i_dag*N_j\
 + H100[p]*P_i_dag + H011[p,q]*N_i*P_j + H001[p]*P_i + H010[p]*N_i + H020[p,q]*N_i*N_j


#expr = H120[p,q]*N[p]*N[q]
#expr = H120[p,q]*N_i*N_j
expr = dr.einst(expr).simplify().merge()
expr.display()

<IPython.core.display.Math object>

In [9]:
Exp = expr.simplify().eval_vev().merge()
Exp.display()

<IPython.core.display.Math object>

In [11]:
print(Exp.latex())

\sum_{p \in A} \left(h^{(001)}_{p} u_{p} v_{p} + 2 h^{(010)}_{p} v_{p}^{2} - 2 h^{(011)}_{p,p} u_{p} v_{p}^{3} + 4 h^{(020)}_{p,p} u_{p}^{2} v_{p}^{2} - 4 h^{(021)}_{p,p,p} u_{p}^{3} v_{p}^{3} + 4 h^{(021)}_{p,p,p} u_{p} v_{p}^{5} + h^{(100)}_{p} u_{p} v_{p} - 2 h^{(110)}_{p,p} u_{p} v_{p}^{3} - 4 h^{(120)}_{p,p,p} u_{p}^{3} v_{p}^{3} + 4 h^{(120)}_{p,p,p} u_{p} v_{p}^{5}\right)   + \sum_{p \in A} \sum_{q \in A} \left(2 h^{(011)}_{p,q} u_{q} v_{p}^{2} v_{q} + 4 h^{(020)}_{p,q} v_{p}^{2} v_{q}^{2} + 4 h^{(021)}_{p,p,q} u_{p}^{2} u_{q} v_{p}^{2} v_{q} - 8 h^{(021)}_{p,q,p} u_{p} v_{p}^{3} v_{q}^{2} + 2 h^{(110)}_{p,q} u_{p} v_{p} v_{q}^{2} - 8 h^{(120)}_{p,p,q} u_{p} v_{p}^{3} v_{q}^{2} + 4 h^{(120)}_{p,q,q} u_{p} u_{q}^{2} v_{p} v_{q}^{2}\right)   + \sum_{p \in A} \sum_{q \in A} \sum_{r \in A} \left(4 h^{(021)}_{p,q,r} u_{r} v_{p}^{2} v_{q}^{2} v_{r} + 4 h^{(120)}_{p,q,r} u_{p} v_{p} v_{q}^{2} v_{r}^{2}\right) 


In [ ]:
import re
from sympy.printing.fortran import FCodePrinter
from drudge import Tensor
from gristmill import FortranPrinter, optimize, get_flop_cost
from sympy import IndexedBase

H000_dr =  Exp
H000 = IndexedBase('E')
expr0 = dr.define(H000,H000_dr)
work_eqn = []
work_eqn.append(dr.define(H000, H000_dr))

print("Original cost:", get_flop_cost(work_eqn, leading=True))
eval_equ0 = optimize(
    [expr0],
    interm_fmt='tau{}',drop_cutoff=2
)

fort = FortranPrinter()
code_obj = fort.doprint(eval_equ0) if hasattr(fort, "doprint") else fort.print(eval_equ0)
code = code_obj.text if hasattr(code_obj, "text") else str(code_obj)


print("Optimized cost:", get_flop_cost(eval_equ0, leading=True))


 

with open('SFS_Ene.f90', 'w') as f:
    f.write(code)
print("The fortran code is printed in SFS_Ene.f90")

In [ ]:
import re
from sympy import IndexedBase
from sympy.printing.pycode import PythonCodePrinter
from gristmill import optimize, get_flop_cost, Printer
# if you need Fortran too:
# from gristmill import FortranPrinter
H000_dr =  Exp
H000 = IndexedBase('E')
expr0 = dr.define(H000,H000_dr)
# ---------------------------------------------------------------------
# 1. Define a Python printer that Gristmill-style code can use
# ---------------------------------------------------------------------
class PythonPrinter(Printer):
    """Minimal printer to emit Python code from optimized SymPy/Gristmill exprs."""
    def __init__(self):
        super().__init__()
        self._printer = PythonCodePrinter()

    def doprint(self, expr):
        return self._printer.doprint(expr)

# ---------------------------------------------------------------------
# 2. Pretend we got an expression from Drudge
#    (you must replace this part with your real drudge expression)
# ---------------------------------------------------------------------
# Example:
#   H000_dr = some SymPy expression you got from drudge
# For the demo, let me make a tiny expression:
 

# In your real code this might look like:
# H000 = IndexedBase('E')
# expr0 = dr.define(H000, H000_dr)
# but here I'll just use H000_dr directly.
 

# Gristmill wants a *list* of work equations
work_eqn = [expr0]

print("Original cost:", get_flop_cost(work_eqn, leading=True))

# ---------------------------------------------------------------------
# 3. Optimize
# ---------------------------------------------------------------------
eval_equ0, report = optimize(
    [expr0],
    interm_fmt='tau{}',
    drop_cutoff=2
)

print("Optimized cost:", get_flop_cost(eval_equ0, leading=True))

# ---------------------------------------------------------------------
# 4. Print as Python
# ---------------------------------------------------------------------
py_printer = PythonPrinter()

# eval_equ0 is a list; print each
lines = []
for eq in eval_equ0:
    # eq is usually a sympy Eq(lhs, rhs) or an Assign-like thing
    code = py_printer.doprint(eq)
    lines.append(code)

python_source = "\n".join(lines)

# ---------------------------------------------------------------------
# 5. Write to file
# ---------------------------------------------------------------------
with open('SFS_Ene.py', 'w') as f:
    f.write("# Auto-generated by Gristmill -> Python\n")
    f.write(python_source)
    f.write("\n")

print("The Python code is printed in SFS_Ene.py")


In [10]:
import re
from sympy.printing.fortran import FCodePrinter
from drudge import Tensor
from gristmill import FortranPrinter, optimize, get_flop_cost
from sympy import IndexedBase

H000_dr =  Exp
H000 = IndexedBase('E')
expr0 = dr.define(H000[p],H000_dr)
work_eqn = []
work_eqn.append(dr.define(H000, H000_dr))

print("Original cost:", get_flop_cost(work_eqn, leading=True))
eval_equ0 = optimize(
    [expr0],
    interm_fmt='tau{}',drop_cutoff=2
)

fort = FortranPrinter()
code_obj = fort.doprint(eval_equ0) if hasattr(fort, "doprint") else fort.print(eval_equ0)
code = code_obj.text if hasattr(code_obj, "text") else str(code_obj)


print("Optimized cost:", get_flop_cost(eval_equ0, leading=True))


 

with open('SFS_Ene.f90', 'w') as f:
    f.write(code)
print("The fortran code is printed in SFS_Ene.f90")

Original cost: na**3
Optimized cost: 5*na**3
The fortran code is printed in SFS_Ene.f90


In [16]:
from drudge import Tensor
#from drudge import IndexRange, Idx
na = symbols('NAO', integer=True, positive=True)
A = Range('A',1, na)     # |A| = na
p, q, r = Idx('p', A), Idx('q', A), Idx('r', A)


u    = Tensor('u',    (A,))
v    = Tensor('v',    (A,))
h001 = Tensor('h001', (A,))
h010 = Tensor('h010', (A,))
h011 = Tensor('h011', (A,A))
h020 = Tensor('h020', (A,A))
h021 = Tensor('h021', (A,A,A))
h110 = Tensor('h110', (A,A))
h120 = Tensor('h120', (A,A,A))

E = Tensor('E', (A,))   # this is H[p]


SympifyError: SympifyError: "cannot sympify object of type <class 'drudge.term.Range'>"